In [ ]:
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle

request_headers = {
"Connection": "keep-alive",
"Content-Length": "18",
"User-Agent": 
"Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; .NET CLR 2.7.58687; SLCC2; Media Center PC 5.0; Zune 3.4; Tablet PC 3.6; InfoPath.3)",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Accept-Language": "en-US,en;q=0.8",
"Cookie":"multivariate_bot=false; D_SID=18.111.93.191:XQebItn+Xbbs4fxri7rWUENSpuNpcQ10POuRhCUaoqA; ahoy_visitor=85c0aa9d-5741-41b0-90a1-66c2407480de; _gat=1; _gat_newTracker=1; __qca=P0-280926842-1448694800387; __uvt=; amplitude_idcrunchbase.com=eyJkZXZpY2VJZCI6ImZlMjZjYmRmLTBjOTMtNGRhYy04ZmFlLTAyMjA1N2M0MzE1ZiIsInVzZXJJZCI6bnVsbH0=; _ga=GA1.2.961691226.1448694800; D_PID=C2974EEC-C295-3B19-881B-3987D348B229; D_IID=A7BBC9FB-58FB-3614-BC42-DC78B9DE46E3; D_UID=176CC3F7-AB55-33C4-934C-F4E77DFD0092; D_HID=3oV+ttwuR6fu7G+eqiUjkdE6kogcj3jL6jqRRPj1Gsw; s_pers=%20s_fid%3D76E928CEE4BFCC75-18BBFE4C3FC0792F%7C1511853228786%3B%20s_getnr%3D1448694828797-New%7C1511766828797%3B%20s_nrgvo%3DNew%7C1511766828800%3B; s_sess=%20s_cc%3Dtrue%3B%20s_sq%3D%3B; uvts=3rLFEsKv57hLKgqX; ahoy_visit=3bfd8d5a-8f74-44ba-b962-9e3ec8223f7e; RT=sl=10&ss=1448694798233&tt=10870&obo=1&bcn=%2F%2F36cc248b.mpstat.us%2F&sh=1448695098621%3D10%3A1%3A10870%2C1448694830134%3D9%3A1%3A10840%2C1448694829092%3D8%3A1%3A10432%2C1448694826377%3D7%3A1%3A8406%2C1448694825821%3D6%3A1%3A8036&dm=crunchbase.com&si=60d1ff9b-5c80-4211-9ed3-061f33fd2b83&r=https%3A%2F%2Fwww.crunchbase.com%2Forganization%2Friskiq&ul=1448695104730"
}


cb_request_headers = {"Host": "www.crunchbase.com",
"Connection": "keep-alive",
"Cache-Control": "max-age=0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"DNT": "1",
"Referer": "https://www.crunchbase.com/organization/riskiq",
"Accept-Encoding": "gzip, deflate, sdch",
"Accept-Language": "en-US,en;q=0.8"}

link = "https://www.crunchbase.com/organization/riskiq"
request = urllib2.Request(link, headers=cb_request_headers)
contents = urllib.urlopen(link).read()

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#cb_data = {}
start = time.time()
org_list = ["riskiq", "pixalate"]
num_orgs = len(org_list)
browser = webdriver.Firefox()
#browser = webdriver.Chrome('/Users/candokevin/Downloads/chromedriver')

for i in range(num_orgs):
    org = org_list[i]
    uri = "https://www.crunchbase.com/organization/" + org
    browser.get(uri)
    wait = WebDriverWait(browser, 30)
    element = wait.until(EC.element_to_be_clickable((By.ID,'anchor-title')))
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")
    fund_table = soup.find_all("div", class_="funding_rounds")
    fund_table_rows = fund_table[0].find_all("tr")    
    description = soup.find_all("div", class_="description-ellipsis")
    cb_data[org] = (description[0].text, [fund_table_rows[i].text for i in range(1, len(fund_table_rows))])
    
    # Timing and saving code
    if i % 1 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(i) / num_orgs) + ' running for ' + str(mins) + ' mins'
        backup(cb_data, i)
    time.sleep(5)
        
browser.quit()